## Query Deployed Vertex AI Agent Engine Agent

This notebook demonstrates how to connect to and query an agent that has been deployed to Vertex AI Agent Engine.

### 1. Install necessary libraries

If you haven't already, you'll need to install the Vertex AI SDK with ADK and Agent Engine support, and python-dotenv.

In [11]:
%pip install google-cloud-aiplatform[adk,agent_engines] python-dotenv

Note: you may need to restart the kernel to use updated packages.


c:\Proyectos\interno\bq-agent\.venv\Scripts\python.exe: No module named pip


### 2. Import libraries and load environment variables

In [1]:
import os
from dotenv import load_dotenv
import vertexai
from vertexai import agent_engines

# Load environment variables from .env file in the parent directory (if this notebook is in bq_agent/)
# Adjust the path if your .env file is elsewhere relative to this notebook.
load_dotenv(dotenv_path='.env') # Assumes .env is in the parent of bq_agent, or use 'bq_agent/.env' if notebook is at root
load_dotenv() # Also try loading from current dir if notebook is at root with .env

True

### 3. Configure Vertex AI Initialization Parameters

These should match the project, location, and staging bucket used for deploying your agent. The `PROJECT_ID` and `LOCATION` are typically loaded from your `.env` file. The `STAGING_BUCKET` is not strictly needed for querying but good practice for consistency if other operations were to be added.

In [3]:
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
LOCATION = os.getenv("GOOGLE_CLOUD_LOCATION")
# STAGING_BUCKET is not directly used for querying a deployed agent but kept for completeness
# STAGING_BUCKET = "gs://your-google-cloud-storage-bucket" # Replace if needed, or load from .env

if not PROJECT_ID or not LOCATION:
    raise ValueError("GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_LOCATION must be set in the environment or .env file.")

print(f"Using Project ID: {PROJECT_ID}")
print(f"Using Location: {LOCATION}")

Using Project ID: dvt-sp-agentspace
Using Location: us-central1


### 4. Initialize Vertex AI

In [4]:
try:
    vertexai.init(
        project=PROJECT_ID,
        location=LOCATION,
        # staging_bucket=STAGING_BUCKET # Not strictly needed for querying
    )
    print("Vertex AI initialized successfully.")
except Exception as e:
    print(f"Error initializing Vertex AI: {e}")
    print("Please ensure you have authenticated with Google Cloud (e.g., `gcloud auth application-default login`) and the Vertex AI API is enabled.")

Vertex AI initialized successfully.


### 5. Get the Deployed Agent

**IMPORTANT:** You need to replace `"YOUR_DEPLOYED_AGENT_RESOURCE_NAME_HERE"` with the actual `resource_name` of your deployed agent. This `resource_name` is printed to the console when you successfully run `python bq_agent/agent.py` (or the script that deploys your agent).

In [10]:
# Replace this with the actual resource name of your deployed agent
DEPLOYED_AGENT_RESOURCE_NAME = "projects/239233954615/locations/us-central1/reasoningEngines/4531650368018317312"

if DEPLOYED_AGENT_RESOURCE_NAME == "projects/YOUR_PROJECT_NUMBER/locations/YOUR_LOCATION/reasoningEngines/YOUR_RESOURCE_ID":
    print("🛑 STOP! Please replace 'DEPLOYED_AGENT_RESOURCE_NAME' with your actual agent's resource name in the cell above. 🛑")
    remote_app = None
else:
    try:
        remote_app = agent_engines.get(DEPLOYED_AGENT_RESOURCE_NAME)
        print(f"Successfully retrieved deployed agent: {remote_app.resource_name if remote_app else 'Not found'}")
    except Exception as e:
        print(f"Error getting deployed agent '{DEPLOYED_AGENT_RESOURCE_NAME}': {e}")
        print("Ensure the resource name is correct and the agent is successfully deployed.")
        remote_app = None

Successfully retrieved deployed agent: projects/239233954615/locations/us-central1/reasoningEngines/4531650368018317312


### 6. Create a Session with the Deployed Agent

In [11]:
remote_session = None
if remote_app:
    try:
        # Use a unique user_id for each user interacting with the agent
        user_id_for_session = "notebook_user_001"
        remote_session = remote_app.create_session(user_id=user_id_for_session)
        print(f"Created remote session: {remote_session}")
    except Exception as e:
        print(f"Error creating remote session: {e}")

Created remote session: {'id': '5264035613035724800', 'appName': '4531650368018317312', 'userId': 'notebook_user_001', 'state': {}, 'events': [], 'lastUpdateTime': 1749568785.541678}


### 7. Send a Query to the Deployed Agent

Now you can send queries to your agent. Modify the `message` variable with your question.

In [12]:
if remote_app and remote_session:
    try:
        message_to_agent = "What tables are available in the dataset?" # Example query
        # You can also ask things like: "Show me the schema for the 'iowa_liquor_sales' table."
        # Or: "What are the first 2 entries in the 'iowa_liquor_sales' table?"
        
        print(f"\nSending query to agent: '{message_to_agent}'")
        print("Agent response stream:")
        for event in remote_app.stream_query(
            user_id=remote_session["userId"],
            session_id=remote_session["id"],
            message=message_to_agent,
        ):
            print(event)
    except Exception as e:
        print(f"Error sending query to agent: {e}")
else:
    print("Cannot send query: remote_app or remote_session is not available. Check previous steps for errors.")


Sending query to agent: 'What tables are available in the dataset?'
Agent response stream:
{'content': {'parts': [{'function_call': {'id': 'adk-48fee473-142d-41db-9956-2b98a1c929f9', 'args': {}, 'name': 'list_dataset_tables'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 5, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 5}], 'prompt_token_count': 725, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 725}], 'thoughts_token_count': 39, 'total_token_count': 769, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-9fa20231-318e-47f9-982b-d1914a42962a', 'author': 'BigQueryAssistant', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'jonCuHOZ', 'timestamp': 1749568787.699061}
{'content': {'parts': [{'function_response': {'id': 'adk-48fee473-142d-41db-9956-2b98a1c929f9', 'name': 'list_dataset_tables', 'response': {'result': ['boilers', 'directorio_empleados', 'directorio_em

### 8. (Optional) Clean up Deployed Agent

If you want to delete the deployed agent from Vertex AI Agent Engine to avoid ongoing charges, you can run the following code. 
**Be careful, as this will permanently delete the agent and its associated resources.**

In [ ]:
# UNCOMMENT THE CODE BELOW TO DELETE THE AGENT
# Make sure DEPLOYED_AGENT_RESOURCE_NAME is correctly set to the agent you want to delete.

# if remote_app:
#     try:
#         print(f"\nAttempting to delete agent: {remote_app.resource_name}")
#         remote_app.delete(force=True) # force=True also deletes child resources like sessions
#         print(f"Successfully initiated deletion of agent: {remote_app.resource_name}")
#     except Exception as e:
#         print(f"Error deleting agent {remote_app.resource_name}: {e}")
# elif DEPLOYED_AGENT_RESOURCE_NAME and DEPLOYED_AGENT_RESOURCE_NAME != "projects/YOUR_PROJECT_NUMBER/locations/YOUR_LOCATION/reasoningEngines/YOUR_RESOURCE_ID":
#     # If remote_app wasn't loaded but we have a resource name, try to get it then delete
#     try:
#         app_to_delete = agent_engines.get(DEPLOYED_AGENT_RESOURCE_NAME)
#         if app_to_delete:
#             print(f"\nAttempting to delete agent: {app_to_delete.resource_name}")
#             app_to_delete.delete(force=True)
#             print(f"Successfully initiated deletion of agent: {app_to_delete.resource_name}")
#         else:
#             print(f"Agent not found for deletion: {DEPLOYED_AGENT_RESOURCE_NAME}")
#     except Exception as e:
#         print(f"Error finding or deleting agent {DEPLOYED_AGENT_RESOURCE_NAME}: {e}")
# else:
#     print("Agent not loaded or resource name not set, cannot delete.")